In [15]:
import tkinter as tk
from tkinter import ttk, messagebox, Toplevel
import utils.gmail_auth
import scraping.latest_emails
from datetime import datetime

class EmailProcessingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Email Processing App")
        self.is_fetching = False
        self.root.configure(bg="#f7f9fc")  # Subtle light background

        # Frame for Gmail Actions
        gmail_frame = tk.LabelFrame(root, text="Gmail Actions", padx=10, pady=10, bg="#ffffff", font=("Arial", 10, "bold"))
        gmail_frame.grid(row=0, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        self.auth_button = tk.Button(gmail_frame, text="Sign In", command=self.gmail_sign_in, bg="#d4edda", fg="#155724", font=("Arial", 10, "bold"), relief="raised")
        self.auth_button.grid(row=0, column=0, padx=5, pady=5)

        self.auth_status_label = tk.Label(gmail_frame, text="Not Signed In", fg="#721c24", bg="#ffffff", font=("Arial", 10))
        self.auth_status_label.grid(row=0, column=1, padx=10, pady=5, sticky="w")

        self.signout_button = tk.Button(gmail_frame, text="Sign Out", command=self.gmail_sign_out, state="disabled", bg="#f8d7da", fg="#721c24", font=("Arial", 10, "bold"), relief="raised")
        self.signout_button.grid(row=0, column=2, padx=5, pady=5)

        # Frame for Email Fetching Statistics
        fetch_frame = tk.LabelFrame(root, text="Email Fetching Statistics", padx=10, pady=10, bg="#ffffff", font=("Arial", 10, "bold"))
        fetch_frame.grid(row=1, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        tk.Label(fetch_frame, text="Daily Emails Fetched:", bg="#ffffff", font=("Arial", 10)).grid(row=0, column=0, sticky="w")
        self.daily_fetch_label = tk.Label(fetch_frame, text="0", bg="#ffffff", font=("Arial", 10, "bold"))
        self.daily_fetch_label.grid(row=0, column=1, sticky="w")

        tk.Label(fetch_frame, text="Total Emails Fetched:", bg="#ffffff", font=("Arial", 10)).grid(row=1, column=0, sticky="w")
        self.total_fetch_label = tk.Label(fetch_frame, text="0", bg="#ffffff", font=("Arial", 10, "bold"))
        self.total_fetch_label.grid(row=1, column=1, sticky="w")

        tk.Label(fetch_frame, text="Failed Emails Fetched:", bg="#ffffff", font=("Arial", 10)).grid(row=2, column=0, sticky="w")
        self.failed_fetch_label = tk.Label(fetch_frame, text="0", bg="#ffffff", font=("Arial", 10, "bold"))
        self.failed_fetch_label.grid(row=2, column=1, sticky="w")

        # Frame for Scraping Statistics
        scrape_frame = tk.LabelFrame(root, text="Scraping Statistics", padx=10, pady=10, bg="#ffffff", font=("Arial", 10, "bold"))
        scrape_frame.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        tk.Label(scrape_frame, text="Daily Scraped Items:", bg="#ffffff", font=("Arial", 10)).grid(row=0, column=0, sticky="w")
        self.daily_scrape_label = tk.Label(scrape_frame, text="0", bg="#ffffff", font=("Arial", 10, "bold"))
        self.daily_scrape_label.grid(row=0, column=1, sticky="w")

        tk.Label(scrape_frame, text="Total Scraped Items:", bg="#ffffff", font=("Arial", 10)).grid(row=1, column=0, sticky="w")
        self.total_scrape_label = tk.Label(scrape_frame, text="0", bg="#ffffff", font=("Arial", 10, "bold"))
        self.total_scrape_label.grid(row=1, column=1, sticky="w")

        tk.Label(scrape_frame, text="Failed Scraped Items:", bg="#ffffff", font=("Arial", 10)).grid(row=2, column=0, sticky="w")
        self.failed_scrape_label = tk.Label(scrape_frame, text="0", bg="#ffffff", font=("Arial", 10, "bold"))
        self.failed_scrape_label.grid(row=2, column=1, sticky="w")

        # Start and Stop Fetching Buttons
        action_frame = tk.LabelFrame(root, text="Actions", padx=10, pady=10, bg="#ffffff", font=("Arial", 10, "bold"))
        action_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        self.start_button = tk.Button(action_frame, text="Start Fetching", command=self.start_fetching, state="disabled", bg="#cce5ff", fg="#004085", font=("Arial", 10, "bold"), relief="raised")
        self.start_button.grid(row=0, column=0, padx=5, pady=5)

        self.stop_button = tk.Button(action_frame, text="Stop Fetching", command=self.stop_fetching, state="disabled", bg="#f8d7da", fg="#721c24", font=("Arial", 10, "bold"), relief="raised")
        self.stop_button.grid(row=0, column=1, padx=5, pady=5)

        # Log Buttons Section
        log_frame = tk.LabelFrame(root, text="Logs", padx=10, pady=10, bg="#ffffff", font=("Arial", 10, "bold"))
        log_frame.grid(row=4, column=0, columnspan=2, padx=10, pady=10, sticky="ew")

        self.logs = {
            "Gmail Authentication": self.create_log_button("Gmail Authentication", log_frame),
            "Email Fetching Status": self.create_log_button("Email Fetching Status", log_frame),
            "Scraping Status": self.create_log_button("Scraping Status", log_frame),
            "Data Consolidation": self.create_log_button("Data Consolidation", log_frame)
        }

    def create_log_button(self, title, parent):
        button = tk.Button(parent, text=title, command=lambda: self.open_log_window(title), bg="#e2e3e5", fg="#383d41", font=("Arial", 10, "bold"), relief="raised")
        button.pack(side="left", padx=5, pady=5)
        return []

    def open_log_window(self, title):
        log_window = Toplevel(self.root)
        log_window.title(title)

        text_widget = ttk.Treeview(log_window, columns=("Time", "Message"), show="headings")
        text_widget.heading("Time", text="Time")
        text_widget.heading("Message", text="Message")
        text_widget.column("Time", width=150, anchor="center")
        text_widget.column("Message", width=600, anchor="w")
        text_widget.pack(side="left", fill="both", expand=True)

        scroll_y = tk.Scrollbar(log_window, orient="vertical", command=text_widget.yview)
        scroll_y.pack(side="right", fill="y")
        text_widget.configure(yscrollcommand=scroll_y.set)

        if title in self.logs:
            for entry in self.logs[title]:
                text_widget.insert("", "end", values=entry)

    def log(self, log_type, message):
        if log_type in self.logs:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            self.logs[log_type].append((timestamp, message))

    def gmail_sign_in(self):
        try:
            self.service = utils.gmail_auth.authenticate_gmail()
            self.auth_status_label.config(text=f"Signed In on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", fg="#155724")
            self.signout_button.config(state="normal")
            self.start_button.config(state="normal")
            self.log("Gmail Authentication", "Successfully signed in to Gmail.")
        except Exception as e:
            self.auth_status_label.config(text="Sign In Failed", fg="#721c24")
            self.log("Gmail Authentication", f"Error during sign in: {e}")
            messagebox.showerror("Error", f"Sign In failed: {e}")

    def gmail_sign_out(self):
        try:
            self.service = None
            self.auth_status_label.config(text="Not Signed In", fg="#721c24")
            self.signout_button.config(state="disabled")
            self.start_button.config(state="disabled")
            self.stop_button.config(state="disabled")
            self.log("Gmail Authentication", "Successfully signed out of Gmail.")
        except Exception as e:
            self.log("Gmail Authentication", f"Error during sign out: {e}")

    def start_fetching(self):
        if not self.is_fetching:
            self.is_fetching = True
            self.start_button.config(state="disabled")
            self.stop_button.config(state="normal")
            self.log("Email Fetching Status", "Fetching started...")
            try:
                self.daily_fetch_count += 10  # Example value
                self.total_fetch_count += 10
                self.failed_fetch_count += 1

                self.daily_scrape_count += 8  # Example value
                self.total_scrape_count += 8
                self.failed_scrape_count += 1

                self.update_statistics()
                self.log("Scraping Status", "Scraping started...")
                self.log("Scraping Status", "Scraping completed successfully!")
                self.log("Data Consolidation", "Data consolidation started...")
                self.log("Data Consolidation", "Data consolidation completed successfully!")
            except Exception as e:
                self.log("Email Fetching Status", f"Error: {e}")

    def stop_fetching(self):
        if self.is_fetching:
            self.is_fetching = False
            self.start_button.config(state="normal")
            self.stop_button.config(state="disabled")
            self.log("Email Fetching Status", "Fetching stopped.")

    def update_statistics(self):
        self.daily_fetch_label.config(text=str(self.daily_fetch_count))
        self.total_fetch_label.config(text=str(self.total_fetch_count))
        self.failed_fetch_label.config(text=str(self.failed_fetch_count))

        self.daily_scrape_label.config(text=str(self.daily_scrape_count))
        self.total_scrape_label.config(text=str(self.total_scrape_count))
        self.failed_scrape_label.config(text=str(self.failed_scrape_count))

if __name__ == "__main__":
    root = tk.Tk()
    app = EmailProcessingApp(root)
    root.mainloop()
